## CostPathFromSurface Class

In [10]:
# Using virtual environment cloned from ArcGIS Pro
import arcpy
import time
import os

In [12]:
class CostPathFromSurface:
    def __init__(self, workSpaceGDB: str, surfaceFile: str, originLoc: str, destLoc: str, siteName: str,
                 inPrj: str, outPrj: str, surfaceAttr: str, rastCellSize: str, ):
        self.workSpaceGDB,self.surfaceFile,self.originLoc,self.destLoc,self.siteName = workSpaceGDB,surfaceFile,originLoc,destLoc,siteName
        self.inPrj,self.outPrj,self.surfaceAttr,self.rastCellSize = inPrj, outPrj, surfaceAttr, rastCellSize
        self.timeSuffix, self.resamplingType, self.prjRastCellSize = time.strftime('%d%m%Y%H%M%S'), 'NEAREST', '45.6326806908288 45.6326806908286'
        # setup arcpy environment
        arcpy.env.workspace, arcpy.env.overwriteOutput = self.workSpaceGDB, True
        self.scratch = arcpy.env.scratchGDB
        # Full function workflow for route generation
        self.surfaceVecToRast()
        self.projectRaster()
        self.projectLocations()
        self.leastCostPath()

    # Convert surface vector into raster format - self.rasterSurface
    def surfaceVecToRast(self):
        self.rasterSurface = (f'{self.siteName}_rasterSurface_{self.timeSuffix}')
        arcpy.conversion.PolygonToRaster(self.surfaceFile, self.surfaceAttr, self.rasterSurface, cell_assignment="CELL_CENTER", 
                                         priority_field="NONE", cellsize=self.rastCellSize, build_rat="BUILD")

    # Project surface raster
    def projectRaster(self):
        self.rastSurfProjName = (f'{self.siteName}_rasterSurfaceProject_{self.timeSuffix}')
        self.rasterSurfaceProject = arcpy.management.ProjectRaster(self.rasterSurface, self.rastSurfProjName, out_coor_system=self.outPrj, 
            resampling_type=self.resamplingType, cell_size=self.prjRastCellSize, geographic_transform=[], Registration_Point="", 
            in_coor_system="", vertical="NO_VERTICAL") 
            
    # Project origin and destination vector points
    def projectLocations(self):
        self.originProj = arcpy.management.Project(self.originLoc, (f'{self.originLoc}_orjProj'), out_coor_system=self.outPrj, transform_method=[], 
            in_coor_system=self.inPrj, preserve_shape="NO_PRESERVE_SHAPE", max_deviation="", vertical="NO_VERTICAL")
        self.destProj = arcpy.management.Project(self.destLoc, (f'{self.destLoc}_orjProj'), out_coor_system=self.outPrj, transform_method=[], 
            in_coor_system=self.inPrj, preserve_shape="NO_PRESERVE_SHAPE", max_deviation="", vertical="NO_VERTICAL")
    
    # And finally generate leastCostPath!
    def leastCostPath(self):
        with arcpy.EnvManager(scratchWorkspace=self.scratch, workspace=self.scratch):
            arcpy.intelligence.LeastCostPath(self.rasterSurfaceProject, self.originProj, self.destProj, 
            (f'{self.workSpaceGDB}\\{self.siteName}_leastCostPath'), "SMALL_POSITIVE")


### CostPathFromSurface Workflow

In [13]:
# define input parameters
workSpaceGDB = 'C:\\Users\\Eric Kerney\\arcgisNotebooks\\leastCost\\Surface_v2.gdb\\Surface.gdb'
surfaceFile, originLoc, destLoc, siteName = 'PendletonOR_SuitabilitySurface', 'TribalCenter', 'Pendleton_Lab', 'pendleOR'
inPrj = 'GEOGCS[\"GCS_WGS_1984\",DATUM[\"D_WGS_1984\",SPHEROID[\"WGS_1984\",6378137.0,298.257223563]],PRIMEM[\"Greenwich\",0.0],UNIT[\"Degree\",0.0174532925199433]]'
outPrj = 'PROJCS[\"US_National_Atlas_Equal_Area\",GEOGCS[\"GCS_Sphere_Clarke_1866_Authalic\",DATUM[\"D_Sphere_Clarke_1866_Authalic\",SPHEROID[\"Sphere_Clarke_1866_Authalic\",6370997.0,0.0]],PRIMEM[\"Greenwich\",0.0],UNIT[\"Degree\",0.0174532925199433]],PROJECTION[\"Lambert_Azimuthal_Equal_Area\"],PARAMETER[\"False_Easting\",0.0],PARAMETER[\"False_Northing\",0.0],PARAMETER[\"Central_Meridian\",-100.0],PARAMETER[\"Latitude_Of_Origin\",45.0],UNIT[\"Meter\",1.0]]'
surfaceAttr, rastCellSize = 'score_v2', '0.0004'
# Runs entire workflow 
costSur = CostPathFromSurface(workSpaceGDB, surfaceFile, originLoc, destLoc, siteName, inPrj, outPrj, surfaceAttr, rastCellSize)

In [9]:
time.strftime('%d%m%Y%H%M%S')

'14032022113212'

## RouteToKML Class Definition

In [14]:
!pip install simplekml -I
import requests
import json
import simplekml

  Using cached simplekml-1.3.6-py3-none-any.whl


In [15]:
# Routing Class: takes input GeoJSON file, get elevation data from CARS, returns formatted KML for customer
# Inputs: geoJSONpath: path to GeoJSON file
# - z_units: feet, ft, meters, m
# - agl: above ground level
# - outName: output kml filename
# - featureType: 'LINESTRING' - GeoJSON feature type, only works with LineString currently
# - in_prj: EPSG prj as 'EPSG:4269'
# - in_type: ellipsoid or orthometric 
class RouteToKML:
    def __init__(self, geoJSONpath: str, z_units: str, agl: float, outName: str, featureType: str='LINESTRING',
                 in_prj: str ='EPSG:4269', in_type: str='ellipsoid'):
        self.inputPath, self.z_units, self.agl, self.featureType = geoJSONpath, z_units, agl, featureType
        self.in_prj, self.in_type, self.outName = in_prj, in_type, outName
        # loadGeoJSON function
        self.loadGeoJSON()
        self.addCARSdata()
        self.addAglGeoJSON()
        self.exportToKML()

    # Load raw GeoJSON file as JSON     
    def loadGeoJSON(self):
        f = open(self.inputPath)
        self.loadedGeoJSON = json.load(f)
        return self.loadedGeoJSON

    # send in memory GeoJSON to CARS
    def addCARSdata(self):
    #def carsRequest(geoJSONagl, in_type='ellipsoid', in_prj='EPSG:4269', z_units='m' ):
        self.geoJSONarr = []
        url = "https://func-data-python-prod-cus.azurewebsites.net/api/carElevator?code=BAzIF4SpjMW6qa8GtFvcOf0tCVvAKA9yaZtIWOW9eQ7V6GMJkzrVqQ=="
        headers = { 'Content-Type': 'application/json'}
        # Iterate through list of GeoJSON features
        for i, feature in enumerate(self.loadedGeoJSON['features']):
            payload = json.dumps({
                "in_type": self.in_type,
                "in_prj": self.in_prj,
                "z_units": self.z_units,
                "geometry": self.loadedGeoJSON['features'][i]['geometry']
            })
            response = requests.request("GET", url, headers=headers, data=payload)
            self.carsGeoJSON = json.loads(response.text)
            # append GeoJSON with CARS attributes to geoJSONarr 
            self.geoJSONarr.append(self.carsGeoJSON)
        return self.geoJSONarr

    # Process CARS GeoJSON into final GeoJSON with altitude, agl, height_above_takeoff
    def addAglGeoJSON(self):
        self.finalGeoJSON = []
        if type(self.geoJSONarr) is list:
            for geoJSON in self.geoJSONarr:
                #self.finalGeoJSON = self.geoJSONarr[0]
                launchHeight = geoJSON['features'][0]['properties']['terrain_WGS84'] 
                for z, feature in enumerate(geoJSON['features']):
                    altitude = feature['properties']['terrain_WGS84'] + self.agl
                    geoJSON['features'][z]['properties']['altitude'] = altitude
                    geoJSON['features'][z]['properties']['AGL'] = self.agl
                    geoJSON['features'][z]['properties']['height_above_takeoff'] = round((self.agl + (feature['properties']['terrain_WGS84'] - launchHeight)), 2)
                self.finalGeoJSON.append(geoJSON)
        return self.finalGeoJSON
    
    # save array of GeoJSON feature collections as KML with formatted attribute table
    def exportToKML(self):
        if type(self.finalGeoJSON) is list:
            for z, geoJSON in enumerate(self.finalGeoJSON):
                self.kml = simplekml.Kml()
                for i, feat in enumerate(geoJSON['features']):
                    pnt = self.kml.newpoint(name=feat['id'],coords=[(feat['geometry']['coordinates'][0],feat['geometry']['coordinates'][1])])
                    style1 = '<body style="margin:0px 0px 0px 0px;overflow:auto;background:#FFFFFF;height:100px">'
                    tableHead = '<table style="width:200px;border:1px solid black;font-family:Arial,Verdana,Times;font-size:14px;text-align:center;border-collapse:collapse;padding:3px 3px 3px 3px">'
                    table2 = '<table style="width:200px;border:1px solid black;font-family:Arial,Verdana,Times;font-size:12px;text-align:left;border-spacing:0px; padding:3px 3px 3px 3px">'
                    trHead = '<tr style="line-height:125%text-align:center;font-weight:bold;background:#9CBCE2;">'
                    trBack = '<tr style="line-height:110%" bgcolor="#D4E4F3">'
                    table_, tr, tr_, td, td_ = '</table>','<tr style="line-height:110%">','</tr>','<td>','</td>'
                    id,units,lon,lat= (f'{feat["id"]}'), (f'{feat["properties"]["units"]}'),(f'{round(feat["geometry"]["coordinates"][0],6)}'),(f'{round(feat["geometry"]["coordinates"][1],6)}')
                    alt, agl, hat = (f'{feat["properties"]["altitude"]}'), (f'{feat["properties"]["AGL"]}'), (f'{feat["properties"]["height_above_takeoff"]}')
                    line1 = (f"{style1}{tableHead}{trHead}{td}ID: {id}  -  UNITS: {units}{td_}{tr_}")
                    line2 = (f"{table2}{tr}{td}LON          {td_}{td}{td_}{td}{lon}{td_}{tr_}")
                    line3 = (f"{trBack}{td}LAT          {td_}{td}{td_}{td}{lat}{td_}{tr_}")
                    line4 = (f"{tr}{td}Flight Altitude {td_}{td}{td_}{td}{alt}{td_}{tr_}")
                    line5 = (f"{trBack}{td}AGL          {td_}{td}{td_}{td}{agl}{td_}{tr_}")
                    line6 = (f"{tr}{td}Height Above Takeoff {td_}{td}{td_}{td}{hat}{td_}{tr_}")
                    pnt.description = (f'{line1}{line2}{line3}{line4}{line5}{line6}')   
                self.kml.save(f'{self.outName}{z}.kml')

    # # unit in either m, meters, ft, feet
    # def addZvalGeoJSON(inputGeoJSON, agl, units):
    #     geoJSONcopy = inputGeoJSON
    #     # check if unit conversion is needed
    #     units = str(units)
    #     agl = (agl*.3048) if units == 'ft' or units == 'feet' else agl
    #     # Loop through each feature in GeoJSON and add agl value to geometry for each coordinate pair
    #     for z, feature in enumerate(geoJSONcopy['features']):
    #         for i, coords in enumerate(feature['geometry']['coordinates']):
    #             newGeom = [coords[0],coords[1],agl]
    #             geoJSONcopy['features'][z]['geometry']['coordinates'][i] = newGeom
    #     # return GeoJSON with agl values added
    #     return geoJSONcopy

### RouteToKML Workflow

In [21]:
inputData = 'SampleRoutes_v2.geojson'
z_units = 'ft'
agl = 400
outputName = 'PendletonRoutes'
routes = RouteToKML(inputData, z_units, agl, outputName)